In [1]:
#@title uploader
file_id = "1VpSCc_ms2dj59bMAAOqCu0OI0jjb-8Hq" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html


from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

# Replace the assignment below with your file ID
# to download a different file.
#
# A file ID looks like: 1gLBqEWEBQDYbKCDigHnUXNTkzl-OslSO

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!unzip {fileId['title']}

widsdatathon2025.zip
Archive:  widsdatathon2025.zip
replace Data Dictionary.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SAMPLE_SUBMISSION.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TEST/TEST_CATEGORICAL.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TEST/TEST_QUANTITATIVE_METADATA.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TRAIN/TRAINING_SOLUTIONS.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.5 MB/s eta 0:00:00


In [1]:

import pandas as pd
import numpy as np

In [2]:
#all datasets
train_quantitative = pd.read_excel('TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx')
train_answers = pd.read_excel('TRAIN/TRAINING_SOLUTIONS.xlsx')
train_categorical = pd.read_excel("TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
train_solutions = pd.read_excel("TRAIN/TRAINING_SOLUTIONS.xlsx")
fcm = pd.read_csv("TRAIN/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv")
fcm_solutions = pd.read_excel("TRAIN/TRAINING_SOLUTIONS.xlsx")
graph_fcm = fcm.merge(fcm_solutions, on="participant_id")

In [11]:
print(train_quantitative.shape)
print(train_categorical.shape)
print(fcm.shape)
print(train_solutions.shape)

(1213, 19)
(1213, 10)
(1213, 19901)
(1213, 3)


In [13]:
train_quantitative.head()

,participant_id,EHQ_EHQ_Total,ColorVision_CV_Score,APQ_P_APQ_P_CP,APQ_P_APQ_P_ID,APQ_P_APQ_P_INV,APQ_P_APQ_P_OPD,APQ_P_APQ_P_PM,APQ_P_APQ_P_PP,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,UmrK0vMLopoR,40.00,13,3,10,47,13,11,28,0,6,1,5,0,5,1,0,10,NaN
1,CPaeQkhcjg7d,-94.47,14,3,13,34,18,23,30,0,18,6,8,7,8,10,4,5,NaN
2,Nb4EetVPm3gs,-46.67,14,4,10,35,16,10,29,1,14,2,8,5,7,6,4,9,8.239904
3,p4vPhVu91o4b,-26.68,10,5,12,39,19,16,28,6,24,4,16,9,10,8,4,6,NaN
4,M09PXs7arQ5E,0.00,14,5,15,40,20,24,28,1,18,4,11,4,10,7,3,9,8.940679


In [14]:
train_categorical.head()

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ
0,UmrK0vMLopoR,2016,1,0.0,0,1,21,45,21,45
1,CPaeQkhcjg7d,2019,3,1.0,2,3,15,15,0,0
2,Nb4EetVPm3gs,2016,1,1.0,8,1,18,40,0,0
3,p4vPhVu91o4b,2018,3,0.0,8,3,15,30,18,0
4,M09PXs7arQ5E,2019,3,0.0,1,3,15,20,0,0


In [20]:
fcm.head()

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,70z8Q2xdTXM3,0.093473,0.146902,0.067893,0.015141,0.070221,0.063997,0.055382,-0.035335,0.068583,...,0.003404,-0.010359,-0.050968,-0.014365,0.128066,0.112646,-0.058980,0.028228,0.133582,0.143372
1,WHWymJu6zNZi,0.029580,0.179323,0.112933,0.038291,0.104899,0.064250,0.008488,0.077505,-0.004750,...,-0.008409,-0.008479,0.020891,0.017754,0.094040,0.035141,0.032537,0.075007,0.115350,0.138200
2,4PAQp1M6EyAo,-0.051580,0.139734,0.068295,0.046991,0.111085,0.026978,0.151377,0.021198,0.083721,...,0.053245,-0.028003,0.028773,0.024556,0.166343,0.058925,0.035485,0.063661,0.042862,0.162162
3,obEacy4Of68I,0.016273,0.204702,0.115980,0.043103,0.056431,0.057615,0.055773,0.075030,0.001033,...,-0.023918,-0.005356,0.018607,0.016193,0.072955,0.130135,0.056120,0.084784,0.114148,0.190584
4,s7WzzDcmDOhF,0.065771,0.098714,0.097604,0.112988,0.071139,0.085607,0.019392,-0.036403,-0.020375,...,0.066439,-0.076680,-0.047530,-0.031443,0.221213,0.007343,0.005763,0.083820,0.079582,0.067269


## Data Prep

In [4]:
#categorical model prep
#fill missing NA values
train_categorical.fillna({'PreInt_Demos_Fam_Child_Ethnicity':train_categorical['PreInt_Demos_Fam_Child_Ethnicity'].mean()}, inplace = True)


In [7]:
#fcm model prep

# import torch
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv
# from torch_geometric.data import Data
# import numpy as np  # Ensure NumPy is imported

# # Check if GPU is available and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Assuming 'graph_fcm' DataFrame is already loaded and preprocessed
# # Remove 'Sex_F' column
# fcm = fcm.drop('Sex_F', axis=1)

# # Prepare data for PyTorch Geometric
# connectivity_columns = [col for col in graph_fcm.columns if col not in ['participant_id', 'ADHD_Outcome']]
# features = graph_fcm[connectivity_columns].values
# labels = graph_fcm['ADHD_Outcome'].values

# # Create an adjacency matrix (replace with actual adjacency if available)
# num_nodes = features.shape[0]  # Ensure correct shape (number of samples, not features)
# adjacency_matrix = np.ones((num_nodes, num_nodes))
# np.fill_diagonal(adjacency_matrix, 0)  # No self-loops

# # Convert to PyTorch tensors and move to device
# data = Data(
#     x=torch.tensor(features, dtype=torch.float32).to(device),
#     edge_index=torch.tensor(np.array(np.where(adjacency_matrix)), dtype=torch.long).to(device),
#     y=torch.tensor(labels, dtype=torch.long).to(device)
# )

## Modeling to predict ADHD_Outcome

In [ ]:
# multi_input_model_colab.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
import gc

# ====== CONFIG ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ====== LOAD DATA (MAKE SURE TO DEFINE THESE IN YOUR NOTEBOOK BEFORE IMPORTING) ======
numerical_df = train_quantitative  # shape (1213, 18)
categorical_df = train_categorical  # shape (1213, N_cat_cols)
graph_fcm_df = fcm  # contains connectivity + ADHD_Outcome

# ====== PREPROCESSING ======
numerical_df = numerical_df.sort_values(by="participant_id").reset_index(drop=True)
categorical_df = categorical_df.sort_values(by="participant_id").reset_index(drop=True)
graph_fcm_df = graph_fcm_df.sort_values(by="participant_id").reset_index(drop=True)
train_solutions = train_solutions.sort_values(by="participant_id").reset_index(drop=True)

assert all(numerical_df['participant_id'] == categorical_df['participant_id'])
assert all(numerical_df['participant_id'] == graph_fcm_df['participant_id'])

labels = train_solutions["ADHD_Outcome"].values
numerical_df = numerical_df.drop(columns=["participant_id"])
categorical_df = categorical_df.drop(columns=["participant_id"])
graph_fcm_df = graph_fcm_df.drop(columns=["participant_id"])

# Normalize numerical features
num_scaler = StandardScaler()
num_features = num_scaler.fit_transform(numerical_df.values)

# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_features = encoder.fit_transform(categorical_df.values)

# Combine tabular features
tabular_features = np.concatenate([num_features, cat_features], axis=1)
print("Tabular input dim:", tabular_features.shape[1])

# Graph features
connectivity_cols = [col for col in graph_fcm_df.columns if col != 'ADHD_Outcome']
graph_features = graph_fcm_df[connectivity_cols].values
num_nodes = graph_features.shape[1]

# Simple fully-connected adjacency
edge_index = torch.tensor(np.array(np.where(np.ones((num_nodes, num_nodes)) - np.eye(num_nodes))), dtype=torch.long)

# Create PyG Data list
data_list = []
for i in range(len(graph_features)):
    graph_x = torch.tensor(graph_features[i].reshape(-1, 1), dtype=torch.float)
    data = Data(
        x=graph_x,
        edge_index=edge_index,
        y=torch.tensor(labels[i], dtype=torch.long),
        tabular=torch.tensor(tabular_features[i], dtype=torch.float),
        sample_idx=i
    )
    data_list.append(data)

# ====== DATA SPLIT & LOADERS ======
train_data, test_data = train_test_split(data_list, test_size=0.2, stratify=labels, random_state=42)
train_loader = DataLoader(train_data, batch_size=1, shuffle=True)  # Reduced batch size
test_loader = DataLoader(test_data, batch_size=1)  # Reduced batch size

# ====== MODEL ======
class MultiInputGCN(nn.Module):
    def __init__(self, tabular_input_dim):
        super().__init__()
        self.gcn1 = GCNConv(1, 16)
        self.gcn2 = GCNConv(16, 32)
        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 + 32, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.gcn1(x, edge_index))
        x = F.relu(self.gcn2(x, edge_index))
        graph_embed = global_mean_pool(x, batch)
        tabular_embed = self.tabular_net(data.tabular)
        combined = torch.cat([graph_embed, tabular_embed], dim=1)
        return self.classifier(combined).squeeze(1)

# ====== TRAINING SETUP ======
model = MultiInputGCN(tabular_input_dim=tabular_features.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.BCELoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        del data
        torch.cuda.empty_cache()  # Clear GPU cache
        gc.collect()
    print(f"Train Loss: {total_loss / len(train_loader):.4f}")

def evaluate(loader, mode="Test"):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data)
            preds = (out > 0.5).long()
            correct += (preds == data.y).sum().item()
            total += len(data.y)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(data.y.cpu().numpy())
            del data
            torch.cuda.empty_cache()  # Clear GPU cache
            gc.collect()
    acc = correct / total
    print(f"{mode} Accuracy: {acc:.4f}")
    return all_preds, all_labels

# ====== TRAIN LOOP ======
for epoch in range(1, 21):
    print(f"\nEpoch {epoch}")
    train()
    evaluate(test_loader)

# ====== SAVE MODEL ======
torch.save(model.state_dict(), "multi_input_model_colab.pth")

# ====== INFERENCE FUNCTION ======
def predict_on_new_data(new_tabular, new_graph):
    model.eval()
    with torch.no_grad():
        graph_x = torch.tensor(new_graph.reshape(-1, 1), dtype=torch.float).to(device)
        tabular_x = torch.tensor(new_tabular, dtype=torch.float).to(device)
        data = Data(x=graph_x, edge_index=edge_index.to(device), tabular=tabular_x)
        data.batch = torch.zeros(data.x.shape[0], dtype=torch.long).to(device)
        out = model(data)
        return int(out.item() > 0.5), out.item()

Using device: cpu
Tabular input dim: 84
